<a href="https://colab.research.google.com/github/AyHaski/BigDataAnalyticsProject/blob/master/Kopie_von_Rapids_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Sun Aug 23 11:06:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
1. removes incompatible files
1. Install RAPIDS libraries
1. Set necessary environment variables
1. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions
1. If running v0.11 or higher, updates pyarrow library to 0.15.x.

In [1]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 51.06 KiB | 475.00 KiB/s, done.
Resolving deltas: 100% (62/62), done.
PLEASE READ
********************************************************************************************************
Changes:
1. Default stable version is now 0.14.  Nightly is now 0.15.  We have fixed the long conda install.  Hooray!
2. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.15, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh s'
                  '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.15, or '!bash rapidsai-csp-utils/col

#[cuDF](https://github.com/rapidsai/cudf)#

Load a dataset into a GPU memory resident DataFrame and perform a basic calculation.

Everything from CSV parsing to calculating tip percentage and computing a grouped average is done on the GPU.

_Note_: You must import nvstrings and nvcategory before cudf, else you'll get errors.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [43]:
import nvstrings
import numpy as np
import cudf, cuml
import io, requests
import math

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns 

#Learning
from cuml.preprocessing.model_selection import train_test_split
from cuml.linear_model import LinearRegression

from cuml.solvers import SGD as cumlSGD
from cuml.ensemble import RandomForestRegressor as cuRF
from cuml.neighbors import KNeighborsRegressor
from cuml import ForestInference
import xgboost as xgb
from cuml.svm import SVC

from sklearn.metrics import accuracy_score
from cuml.metrics.regression import r2_score


In [4]:
cudf.set_allocator("managed")
base_path='/content/drive/My Drive/bigData'

traintypes = {'fare_amount': 'float32',
              'pickup_datetime':'str',
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'int8'}

cols = list(traintypes.keys())
usecols = cols

In [5]:
%%time
gdf_test = cudf.read_csv(base_path +'/test.csv', usecols=cols, dtype=traintypes)
gdf_train = cudf.read_csv(base_path +'/train.csv', usecols=cols, dtype=traintypes)

CPU times: user 3.23 s, sys: 3.12 s, total: 6.35 s
Wall time: 1min 56s


In [9]:
gdf_train.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844307,40.721321,-73.841614,40.712273,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016045,40.711304,-73.979271,40.782005,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982742,40.761269,-73.991234,40.750565,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987137,40.733139,-73.991570,40.758095,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968102,40.768009,-73.956665,40.783768,1


In [10]:
gdf_train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542386e+07,5.542386e+07,5.542386e+07,5.542348e+07,5.542348e+07,5.542386e+07
mean,1.134505e+01,-7.250969e+01,3.991979e+01,-7.251121e+01,3.992068e+01,1.685075e+00
std,2.071083e+01,1.284888e+01,9.642353e+00,1.278220e+01,9.633346e+00,1.310116e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442024e+03,-3.547886e+03,-1.270000e+02
25%,6.000000e+00,-7.399206e+01,4.073493e+01,-7.399139e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398016e+01,4.075315e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396368e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457625e+03,3.408790e+03,3.457622e+03,3.537133e+03,5.100000e+01


In [6]:
gdf_train['pickup_datetime'] = gdf_train['pickup_datetime'].astype('datetime64[ns]')
gdf_test['pickup_datetime'] = gdf_test['pickup_datetime'].astype('datetime64[ns]')

In [7]:
#Getting interger numbers from the pickup_datetime
gdf_train["hour"] = gdf_train.pickup_datetime.dt.hour
gdf_train["weekday"] = gdf_train.pickup_datetime.dt.weekday
gdf_train["month"] = gdf_train.pickup_datetime.dt.month
gdf_train["year"] = gdf_train.pickup_datetime.dt.year

gdf_test["hour"] = gdf_test.pickup_datetime.dt.hour
gdf_test["weekday"] = gdf_test.pickup_datetime.dt.weekday
gdf_test["month"] = gdf_test.pickup_datetime.dt.month
gdf_test["year"] = gdf_test.pickup_datetime.dt.year

In [13]:
gdf_train.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,weekday,month,year
0,4.5,2009-06-15 17:26:21,-73.844307,40.721321,-73.841614,40.712273,1,17,0,6,2009
1,16.9,2010-01-05 16:52:16,-74.016045,40.711304,-73.979271,40.782005,1,16,1,1,2010
2,5.7,2011-08-18 00:35:00,-73.982742,40.761269,-73.991234,40.750565,2,0,3,8,2011
3,7.7,2012-04-21 04:30:42,-73.987137,40.733139,-73.991570,40.758095,1,4,5,4,2012
4,5.3,2010-03-09 07:51:00,-73.968102,40.768009,-73.956665,40.783768,1,7,1,3,2010


#Data Analysis

Look at missing values in data
Look at what values are in fare_amount and passenger_count
-> distribution graph
Look at coordinates 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#Data Cleaning


In [8]:
print("Shape of Training Data after dropping columns",gdf_train.shape)
print("Shape of Testing Data after dropping columns",gdf_test.shape)

Shape of Training Data after dropping columns (55423856, 11)
Shape of Testing Data after dropping columns (9914, 10)


In [9]:
gdf_train.nans_to_nulls()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,weekday,month,year
0,4.5,2009-06-15 17:26:21,-73.844307,40.721321,-73.84161377,40.71227264,1,17,0,6,2009
1,16.9,2010-01-05 16:52:16,-74.016045,40.711304,-73.97927094,40.78200531,1,16,1,1,2010
2,5.7,2011-08-18 00:35:00,-73.982742,40.761269,-73.99123383,40.75056458,2,0,3,8,2011
3,7.7,2012-04-21 04:30:42,-73.987137,40.733139,-73.99156952,40.75809479,1,4,5,4,2012
4,5.3,2010-03-09 07:51:00,-73.968102,40.768009,-73.95666504,40.7837677,1,7,1,3,2010
...,...,...,...,...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00,-74.005264,40.740028,-73.96327209,40.76255417,1,3,5,3,2014
55423852,4.2,2009-03-24 20:46:20,-73.957794,40.765533,-73.9516449,40.7739563,1,20,1,3,2009
55423853,14.1,2011-04-02 22:04:24,-73.970512,40.752323,-73.96054077,40.79734421,1,22,5,4,2011
55423854,28.9,2011-10-26 05:57:51,-73.980904,40.764629,-73.8706131,40.77396393,1,5,2,10,2011


In [10]:
test_1 = gdf_test
train_1 = gdf_train.dropna()
train_1

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,weekday,month,year
0,4.5,2009-06-15 17:26:21,-73.844307,40.721321,-73.841614,40.712273,1,17,0,6,2009
1,16.9,2010-01-05 16:52:16,-74.016045,40.711304,-73.979271,40.782005,1,16,1,1,2010
2,5.7,2011-08-18 00:35:00,-73.982742,40.761269,-73.991234,40.750565,2,0,3,8,2011
3,7.7,2012-04-21 04:30:42,-73.987137,40.733139,-73.991570,40.758095,1,4,5,4,2012
4,5.3,2010-03-09 07:51:00,-73.968102,40.768009,-73.956665,40.783768,1,7,1,3,2010
...,...,...,...,...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00,-74.005264,40.740028,-73.963272,40.762554,1,3,5,3,2014
55423852,4.2,2009-03-24 20:46:20,-73.957794,40.765533,-73.951645,40.773956,1,20,1,3,2009
55423853,14.1,2011-04-02 22:04:24,-73.970512,40.752323,-73.960541,40.797344,1,22,5,4,2011
55423854,28.9,2011-10-26 05:57:51,-73.980904,40.764629,-73.870613,40.773964,1,5,2,10,2011


In [11]:
train_1.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,weekday,month,year
count,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07
mean,1.134501e+01,-7.250973e+01,3.991982e+01,-7.251121e+01,3.992068e+01,1.685087e+00,1.350978e+01,3.041161e+00,6.266239e+00,2.011739e+03
std,2.071087e+01,1.284877e+01,9.642324e+00,1.278220e+01,9.633346e+00,1.310113e+00,6.517377e+00,1.948911e+00,3.435531e+00,1.860079e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442024e+03,-3.547886e+03,-1.270000e+02,0.000000e+00,0.000000e+00,1.000000e+00,2.009000e+03
25%,6.000000e+00,-7.399206e+01,4.073493e+01,-7.399139e+01,4.073403e+01,1.000000e+00,9.000000e+00,1.000000e+00,3.000000e+00,2.010000e+03
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398016e+01,4.075315e+01,1.000000e+00,1.400000e+01,3.000000e+00,6.000000e+00,2.012000e+03
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396368e+01,4.076810e+01,2.000000e+00,1.900000e+01,5.000000e+00,9.000000e+00,2.013000e+03
max,9.396336e+04,3.457625e+03,3.408790e+03,3.457622e+03,3.537133e+03,5.100000e+01,2.300000e+01,6.000000e+00,1.200000e+01,2.015000e+03


In [12]:
#Check fare_amount im minus macht kein sinn, wird gedroppt
train_1 = train_1[train_1['fare_amount'] > 0]
#Manche Koordinaten sind supper weit weg werden entfernt
train_1 = train_1[(train_1['pickup_longitude'] < -70) & (train_1['pickup_longitude'] > -75)]
train_1 = train_1[(train_1['pickup_latitude'] > 38) & (train_1['pickup_latitude'] < 44)]
train_1 = train_1[(train_1['dropoff_longitude'] < -70) & (train_1['dropoff_longitude'] > -75)]
train_1 = train_1[(train_1['dropoff_latitude'] > 38) & (train_1['dropoff_latitude'] < 44)]
train_1 = train_1[(train_1['passenger_count'] > 0) & (train_1['passenger_count'] < 6)]
train_1.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,weekday,month,year
count,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07,5.407243e+07
mean,1.133485e+01,-7.397512e+01,4.075096e+01,-7.397428e+01,4.075130e+01,1.691095e+00,1.351033e+01,3.041093e+00,6.269149e+00,2.011738e+03
std,2.086523e+01,4.024100e-02,3.260000e-02,3.946200e-02,3.574000e-02,1.306911e+00,6.516747e+00,1.949117e+00,3.436407e+00,1.865444e+00
min,1.000000e-02,-7.499804e+01,3.803999e+01,-7.499828e+01,3.803333e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.009000e+03
25%,6.000000e+00,-7.399227e+01,4.073656e+01,-7.399159e+01,4.073557e+01,1.000000e+00,9.000000e+00,1.000000e+00,3.000000e+00,2.010000e+03
50%,8.500000e+00,-7.398210e+01,4.075337e+01,-7.398061e+01,4.075386e+01,1.000000e+00,1.400000e+01,3.000000e+00,6.000000e+00,2.012000e+03
75%,1.250000e+01,-7.396830e+01,4.076756e+01,-7.396533e+01,4.076840e+01,2.000000e+00,1.900000e+01,5.000000e+00,9.000000e+00,2.013000e+03
max,9.396336e+04,-7.000039e+01,4.399612e+01,-7.000227e+01,4.399612e+01,9.000000e+00,2.300000e+01,6.000000e+00,1.200000e+01,2.015000e+03


#Different distance calculations

## Separate distance positiv

In [ ]:
train_1['longitude_distance']=(train_1['dropoff_longitude']  - train_1['pickup_longitude']).abs()
train_1['latitude_distance']=(train_1['dropoff_latitude'] - train_1['pickup_latitude']).abs()

test_1['longitude_distance']=(test_1['dropoff_longitude'] - test_1['pickup_longitude']).abs()
test_1['latitude_distance']=(test_1['dropoff_latitude'] - test_1['pickup_latitude']).abs()

## Complete distance


### Distance in miles

In [13]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [14]:
train_1['distance_miles'] = distance(train_1['pickup_latitude'], train_1['pickup_longitude'], \
                                      train_1['dropoff_latitude'], train_1['dropoff_longitude'] )

In [15]:
test_1['distance_miles'] = distance(test_1['pickup_latitude'], test_1['pickup_longitude'], \
                                      test_1['dropoff_latitude'], test_1['dropoff_longitude'] )

In [16]:
train_1['distance_miles'].describe()

count    5.407243e+07
mean     1.871327e+00
std      2.649232e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.366828e+00
75%      2.424860e+00
max      2.928418e+02
Name: distance_miles, dtype: float64

# Dropping Columns not needed anymore


In [17]:
drop_columns = ['pickup_datetime','dropoff_latitude','pickup_latitude','dropoff_longitude','pickup_longitude']
train_1=train_1.drop(drop_columns,axis=1)
test_1=test_1.drop(drop_columns,axis=1)

In [18]:
train_1.head()

,fare_amount,passenger_count,hour,weekday,month,year,distance_miles
0,4.5,1,17,0,6,2009,0.000000
1,16.9,1,16,1,1,2010,5.244397
2,5.7,2,0,3,8,2011,0.000000
3,7.7,1,4,5,4,2012,1.932986
4,5.3,1,7,1,3,2010,1.366828


In [19]:
test_1.head()

,passenger_count,hour,weekday,month,year,distance_miles
0,1,13,1,1,2015,1.366828
1,1,13,1,1,2015,1.366828
2,1,11,5,10,2011,0.000000
3,1,21,5,12,2012,1.366828
4,1,21,5,12,2012,3.226874


# Linear Regression

In [22]:
X=train_1.drop(['fare_amount'],axis=1)
y=train_1['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Number of records in training data  43257944
Number of records in validation data  10814485
(43257944, 6)
(10814485, 6)
(43257944,)
(10814485,)


In [20]:
lm = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "eig")
lm.fit(X_train,y_train)
y_pred=lm.predict(X_test)
lm_rmse = r2_score(y_pred, y_test)
print("RMSE for Linear Regression is ",lm_rmse)

RMSE for Linear Regression is  0.09070217609405518


In [21]:
y_pred=lm.predict(test_1)
y_pred

0        9.783966
1        9.783966
2        5.478973
3        9.694916
4       14.977570
          ...    
9909    10.014099
9910    12.425690
9911    39.885895
9912    20.667664
9913     5.894531
Length: 9914, dtype: float32

In [22]:
gdf_submission = cudf.read_csv(base_path +'/sample_submission.csv')
gdf_submission['fare_amount']= y_pred

gdf_submission.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,9.783966
1,2015-01-27 13:08:24.0000003,9.783966
2,2011-10-08 11:53:44.0000002,5.478973
3,2012-12-01 21:12:12.0000002,9.694916
4,2012-12-01 21:12:12.0000003,14.977570


In [23]:
gdf_submission.to_csv(base_path + '/logRegRapidSubmissionMilesDistance3.csv',index=False)

# Random Forest Regression Model

In [52]:
cu_rf_params = {
    'n_estimators': 100,
    'max_depth': 16,
    'n_bins': 8,
}

In [53]:
X=train_1.drop(['fare_amount'],axis=1)
y=train_1['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)


In [54]:
cu_rf = cuRF(**cu_rf_params)
cu_rf.fit(X_train, y_train)
y_pred=cu_rf.predict(X_test)

In [55]:
rf_rmse = r2_score(y_pred, y_test)
print("RMSE for RF Regression is ",rf_rmse)

RMSE for RF Regression is  0.265926718711853


In [56]:
y_pred=cu_rf.predict(test_1)
y_pred

0        9.787024
1        9.787024
2        5.867246
3        8.939384
4       15.635692
          ...    
9909     8.973536
9910    11.656147
9911    34.585255
9912    27.609804
9913     6.255931
Length: 9914, dtype: float32

In [57]:
gdf_submission = cudf.read_csv(base_path +'/sample_submission.csv')
gdf_submission['fare_amount']= y_pred

gdf_submission.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,9.787024
1,2015-01-27 13:08:24.0000003,9.787024
2,2011-10-08 11:53:44.0000002,5.867246
3,2012-12-01 21:12:12.0000002,8.939384
4,2012-12-01 21:12:12.0000003,15.635692


In [58]:
gdf_submission.to_csv(base_path + '/randomForestSubmissionDefaultParams.csv',index=False)

# XGBoost


In [41]:
X=train_1.drop(['fare_amount'],axis=1)
y=train_1['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [44]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidation = xgb.DMatrix(X_test, label=y_test)

In [45]:
# instantiate params
params = {}

# general params
general_params = {'silent': 1}
params.update(general_params)

# booster params
n_gpus = 1  # change this to -1 to use all GPUs available or 0 to use the CPU
booster_params = {}

if n_gpus != 0:
    booster_params['tree_method'] = 'gpu_hist'
    booster_params['n_gpus'] = n_gpus   
params.update(booster_params)

# learning task params
learning_task_params = {}
learning_task_params['eval_metric'] = 'rmse'
learning_task_params['objective'] = 'reg:squarederror'
params.update(learning_task_params)
print(params)

{'silent': 1, 'tree_method': 'gpu_hist', 'n_gpus': 1, 'eval_metric': 'rmse', 'objective': 'reg:squarederror'}


In [46]:
# model training settings
evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
num_round = 100

In [ ]:
bst = xgb.train(params, dtrain, num_round, evallist)

In [49]:
dtest = xgb.DMatrix(test_1)
y_pred_xgb = bst.predict(dtest)
print(y_pred_xgb)

[13:37:28] WARNING: /conda/conda-bld/xgboost_1591204502249/work/include/xgboost/generic_parameters.h:35: 
n_gpus: 
	Deprecated. Single process multi-GPU training is no longer supported.
	Please switch to distributed training with one process per GPU.
	This can be done using Dask or Spark.  See documentation for details.
[ 9.831148   9.831148   5.6229563 ... 56.161964  22.351265   5.890768 ]


In [51]:
gdf_submission = cudf.read_csv(base_path +'/sample_submission.csv')
gdf_submission['fare_amount']= y_pred_xgb

gdf_submission.to_csv(base_path + '/XGBBoostSubmission.csv',index=False)
gdf_submission.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,9.831148
1,2015-01-27 13:08:24.0000003,9.831148
2,2011-10-08 11:53:44.0000002,5.622956
3,2012-12-01 21:12:12.0000002,9.118807
4,2012-12-01 21:12:12.0000003,16.048931


 # SVM Regression Model (SVC)

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib